In [9]:
import requests
from requests import HTTPError,JSONDecodeError
from pprint import pprint

def get_aqi()->list[dict]:
    try:
        url = 'https://data.moenv.gov.tw/api/v2/aqx_p_432?api_key=af57253c-e838-46da-a1f5-12b43afd75f3&limit=1000&sort=ImportDate desc&format=JSON'
        response = requests.get(url,verify=False)
        response.raise_for_status()
        data_list:list= response.json()['records'] #data_list是每一個站的的資料
        return data_list
    except HTTPError as e:
        raise Exception(f"HTTPError:{e}")
    except JSONDecodeError:
        raise Exception(f"內容不是json的格式")
    except Exception:
        raise Exception("不知的錯誤")

def filter_data(site_list:list[dict])->list[dict]:
    '''過濾資料,只要sitename,country,aqi,status'''
    filter_data:list = []
    for site in site_list:
        new_site:dict = dict()
        new_site["站點名稱"] = site['sitename']
        new_site["城市名稱"] = site['county']
        new_site['aqi'] = site['aqi']
        new_site['狀態'] = site['status']
        filter_data.append(new_site)
    return filter_data

    

def main():
    # 下載資料
    try:
        site_list = get_aqi()
    except Exception as e:
        print(e)
        return  #如果在function出錯了,直接return,'提早離開'
    else:
        print("下載成功")

    # 過濾資料
    sites = filter_data(site_list)     
    pprint(sites)

if __name__ == "__main__":
    main()


/opt/conda/lib/python3.13/site-packages/urllib3/connectionpool.py:1097: InsecureRequestWarning: Unverified HTTPS request is being made to host 'data.moenv.gov.tw'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#tls-warnings
  warnings.warn(


下載成功
[{'aqi': '55', '城市名稱': '基隆市', '狀態': '普通', '站點名稱': '基隆'},
 {'aqi': '64', '城市名稱': '新北市', '狀態': '普通', '站點名稱': '汐止'},
 {'aqi': '57', '城市名稱': '新北市', '狀態': '普通', '站點名稱': '新店'},
 {'aqi': '56', '城市名稱': '新北市', '狀態': '普通', '站點名稱': '土城'},
 {'aqi': '65', '城市名稱': '新北市', '狀態': '普通', '站點名稱': '板橋'},
 {'aqi': '63', '城市名稱': '新北市', '狀態': '普通', '站點名稱': '新莊'},
 {'aqi': '57', '城市名稱': '新北市', '狀態': '普通', '站點名稱': '菜寮'},
 {'aqi': '61', '城市名稱': '新北市', '狀態': '普通', '站點名稱': '林口'},
 {'aqi': '58', '城市名稱': '新北市', '狀態': '普通', '站點名稱': '淡水'},
 {'aqi': '53', '城市名稱': '臺北市', '狀態': '普通', '站點名稱': '士林'},
 {'aqi': '65', '城市名稱': '臺北市', '狀態': '普通', '站點名稱': '中山'},
 {'aqi': '58', '城市名稱': '臺北市', '狀態': '普通', '站點名稱': '萬華'},
 {'aqi': '55', '城市名稱': '臺北市', '狀態': '普通', '站點名稱': '古亭'},
 {'aqi': '67', '城市名稱': '臺北市', '狀態': '普通', '站點名稱': '松山'},
 {'aqi': '68', '城市名稱': '臺北市', '狀態': '普通', '站點名稱': '大同'},
 {'aqi': '56', '城市名稱': '桃園市', '狀態': '普通', '站點名稱': '桃園'},
 {'aqi': '61', '城市名稱': '桃園市', '狀態': '普通', '站點名稱': '大園'},
 {'aqi': '60', '城市名稱': '桃園